In [1]:
!pip install transformers
!pip install datasets
!pip install evaluate
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, get_scheduler
from datasets import load_dataset
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm
import evaluate

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


2023-04-11 16:01:06.777602: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 16:01:07.695820: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-11 16:01:07.695873: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-11 16:01:07.695889: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
raw_train_dataset = load_dataset('csv', data_files='cople_ortho.csv', split='train[:80%]')
raw_validation_dataset = load_dataset('csv', data_files='cople_ortho.csv', split='train[80%:90%]')
raw_test_dataset = load_dataset('csv', data_files='cople_ortho.csv', split='train[90%:]')
labels = pd.read_csv('cople_ortho.csv')['Proficiency'].unique().tolist()
print(labels)

Using custom data configuration default-697819366a407adc
Found cached dataset csv (/home/joao/.cache/huggingface/datasets/csv/default-697819366a407adc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Using custom data configuration default-697819366a407adc
Found cached dataset csv (/home/joao/.cache/huggingface/datasets/csv/default-697819366a407adc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)
Using custom data configuration default-697819366a407adc
Found cached dataset csv (/home/joao/.cache/huggingface/datasets/csv/default-697819366a407adc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


['A2', 'B1', 'B2', 'A1', 'C1']


In [3]:
checkpoint = 'neuralmind/bert-base-portuguese-cased'
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=len(labels))
tokenizer = AutoTokenizer.from_pretrained(checkpoint, do_lower_case=False)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

In [5]:
checkpoint = torch.load("model_4_acc86%")
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [6]:
print(raw_train_dataset)
print(raw_validation_dataset)
print(raw_test_dataset)

Dataset({
    features: ['Id', 'Proficiency', 'Text'],
    num_rows: 816
})
Dataset({
    features: ['Id', 'Proficiency', 'Text'],
    num_rows: 102
})
Dataset({
    features: ['Id', 'Proficiency', 'Text'],
    num_rows: 102
})


In [24]:
df = pd.read_csv("cople_ortho.csv")
training_df = df[:816]
validation_df = df[816:816+102]
test_df = df[-102:]

Training dataframe:              Id Proficiency                                               Text
0  pl006CVETI_2          A2  Bom dia,\n\neu queria reclamar [Fig1] o meu te...
1    es019CAETF          A2  Eu gostaria de fazer uma reunião de amigos num...
2    de007CVMTD          B1  Olá, Nuno,\n\nAs minhas férias são um espectác...
3    de016CVATF          B2  Durante um debate ou uma simples conversa num ...
4    it026CAETF          A2  Normalmente não faço muitos planos para o ano ...
Validation dataframe:                Id Proficiency  \
816    fr024CVETI          A2   
817    en037CVETI          A2   
818    it001CVETF          A2   
819  ru007CAATF_2          B2   
820  zh031CAMTD_2          B1   

                                                  Text  
816  Lisboa, 19.09.2011\n\nQuerida FF,\n\nFaz 3 sem...  
817  Quando Roberto chegou a casa ontem, estava mui...  
818  Infelizmente, em Itália temos muitos problemas...  
819  Sebastianismo: o rei D. Sebastião, " Desejado"...  


In [30]:
print(f"Total class distribution:\n{df.value_counts('Proficiency', sort=False)}")
print(f"Train class distribution:\n{training_df.value_counts('Proficiency', sort=False)}")
print(f"Validation class distribution:\n{validation_df.value_counts('Proficiency', sort=False)}")
print(f"Test class distribution:\n{test_df.value_counts('Proficiency', sort=False)}")

Total class distribution:
Proficiency
A1     92
A2    399
B1    292
B2    199
C1     38
dtype: int64
Train class distribution:
Proficiency
A1     77
A2    315
B1    236
B2    160
C1     28
dtype: int64
Validation class distribution:
Proficiency
A1     7
A2    43
B1    27
B2    18
C1     7
dtype: int64
Test class distribution:
Proficiency
A1     8
A2    41
B1    29
B2    21
C1     3
dtype: int64


In [7]:
def class_string_to_int(s):
    if s == 'A1':
        return 0
    if s == 'A2':
        return 1
    if s == 'B1':
        return 2
    if s == 'B2':
        return 3
    if s == 'C1':
        return 4
    if s == 'C2':
        return 5

In [8]:
def tokenize_function(example):
    example['labels'] = [class_string_to_int(proficiency) for proficiency in example['Proficiency']]
    return tokenizer(example["Text"], truncation=True, max_length=512)


tokenized_train_datasets = raw_train_dataset.map(tokenize_function, batched=True)
tokenized_validation_datasets = raw_validation_dataset.map(tokenize_function, batched=True)
tokenized_test_datasets = raw_test_dataset.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Loading cached processed dataset at /home/joao/.cache/huggingface/datasets/csv/default-697819366a407adc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-647d7d12b977d667.arrow
Loading cached processed dataset at /home/joao/.cache/huggingface/datasets/csv/default-697819366a407adc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-993bbb55c8e2030c.arrow
Loading cached processed dataset at /home/joao/.cache/huggingface/datasets/csv/default-697819366a407adc/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-0cb2513695af1ffd.arrow


In [9]:
print(tokenized_train_datasets)
print(tokenized_validation_datasets)
print(tokenized_test_datasets)

Dataset({
    features: ['Id', 'Proficiency', 'Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 816
})
Dataset({
    features: ['Id', 'Proficiency', 'Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 102
})
Dataset({
    features: ['Id', 'Proficiency', 'Text', 'labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 102
})


In [10]:
tokenized_train_datasets = tokenized_train_datasets.remove_columns(["Text", "Id", "Proficiency"])
tokenized_train_datasets.set_format("torch")
tokenized_train_datasets.column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [11]:
tokenized_validation_datasets = tokenized_validation_datasets.remove_columns(["Text", "Id", "Proficiency"])
tokenized_validation_datasets.set_format("torch")
tokenized_validation_datasets.column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [12]:
tokenized_test_datasets = tokenized_test_datasets.remove_columns(["Text", "Id", "Proficiency"])
tokenized_test_datasets.set_format("torch")
tokenized_test_datasets.column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [13]:
train_dataloader = DataLoader(
    tokenized_train_datasets, shuffle=True, batch_size=1, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_validation_datasets, batch_size=1, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_test_datasets, batch_size=1, collate_fn=data_collator
)

In [14]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'labels': torch.Size([1]),
 'input_ids': torch.Size([1, 186]),
 'token_type_ids': torch.Size([1, 186]),
 'attention_mask': torch.Size([1, 186])}

In [15]:
outputs = model(**batch)
print(outputs.loss, outputs.logits.shape)

tensor(0.0102, grad_fn=<NllLossBackward0>) torch.Size([1, 5])


In [16]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

In [17]:
num_epochs = 1000
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)

816000


In [18]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [19]:
accuracy = evaluate.load('accuracy')
recall = evaluate.load('recall')
precision = evaluate.load('precision')
f1 = evaluate.load('f1')

In [ ]:
progress_bar = tqdm(range(num_training_steps))
train_loss_df = []
valid_loss_df = []
patience = 10
min_delta = 0.5
counter = 0
min_valid_loss = np.inf

if hasattr(torch.cuda, 'empty_cache'):
            torch.cuda.empty_cache()

model.train()
for epoch in range(num_epochs):
    train_loss = 0.0
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        train_loss += loss.item()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)
        if hasattr(torch.cuda, 'empty_cache'):
            torch.cuda.empty_cache()
    train_loss /= len(train_dataloader)
    valid_loss = 0.0
    model.eval()
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        valid_loss += loss.item()
        if hasattr(torch.cuda, 'empty_cache'):
            torch.cuda.empty_cache()
    valid_loss /= len(eval_dataloader)
    train_loss_df.append(train_loss)
    valid_loss_df.append(valid_loss)
    if valid_loss < min_valid_loss:
        print(f"Epoch {epoch} - Train loss: {train_loss} - Validation loss {valid_loss}")
        min_valid_loss = valid_loss
        counter = 0
        model_path = 'model_{}'.format(epoch)
        torch.save(model.state_dict(), model_path)
    elif valid_loss > (min_valid_loss + min_delta):
        counter += 1
        if counter >= patience:
            print(f"Early stopping:\nEpoch {epoch} - Train loss: {train_loss} - Validation loss {valid_loss}")
            break
    model.train()

In [ ]:
print(train_loss_df)
print(valid_loss_df)

In [31]:
accuracy = evaluate.load('accuracy')
recall = evaluate.load('recall')
precision = evaluate.load('precision')
f1 = evaluate.load('f1')
preds = []
model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    preds.append(predictions)
    accuracy.add_batch(predictions=predictions, references=batch["labels"])
    recall.add_batch(predictions=predictions, references=batch["labels"])
    precision.add_batch(predictions=predictions, references=batch["labels"])
    f1.add_batch(predictions=predictions, references=batch["labels"])

print(accuracy.compute())
print(recall.compute(average="weighted"))
print(precision.compute(average="weighted"))
print(f1.compute(average="weighted"))

{'accuracy': 0.8627450980392157}
{'recall': 0.8627450980392157}
{'precision': 0.8792557932263815}
{'f1': 0.8640753147118625}


In [34]:
# preds = [x.item() for x in preds]

In [47]:
# preds_df = test_df
# preds_df["labels"] = class_string_to_int()...
# preds_df.drop(['Proficiency', 'Text'], axis=1, inplace=True)
# preds_df["predictions"] = preds
# preds_df.to_csv("best_model_preds_last_10%.csv")

In [ ]:
path = "./third_model_1000_epochs"
torch.save(model.state_dict(), path)